# Advanced Tutorial on MetaSynth

In this tutorial, we will be creating a `MetaFrame`, which is a metadata representation of a given dataset, and proceed by generating synthetic data from it. In the process, we are going to walk through some of the advanced abilities of MetaSynth, such as handling dates, setting distributions and ensuring uniqueness in columns. This example workflow starts from a `.csv` file as input, but it easily adapted to other formats. 

## Step 0: Install the metasynth package and import required packages

In [1]:
# uncomment the following line and run the cell to install metasyn
# %pip install metasyn

In [2]:
# import required packages
import datetime as dt
import polars as pl
from metasyn import MetaFrame, demo_file

## Step 1: Transforming your data into a polars DataFrame

The first step in creating the MetaFrame is reading and converting your dataset to a polars DataFrame. 

In [3]:
# get the path of the demo csv
demo_file_path = demo_file()

# read the data with the correct categorical variables
data_types={
    "Sex": pl.Categorical,
    "Embarked": pl.Categorical
}

df = pl.read_csv(demo_file_path, try_parse_dates=True, dtypes=data_types)

# check out the data
df.head()

PassengerId,Name,Sex,Age,Parch,Ticket,Fare,Cabin,Embarked,Birthday,Board time,Married since,all_NA
i64,str,cat,i64,i64,str,f64,str,cat,date,time,datetime[μs],str
1,"""Braund, Mr. Ow…","""male""",22,0,"""A/5 21171""",7.25,null,"""S""",1937-10-28,15:53:04,2022-08-05 04:43:34,null
2,"""Cumings, Mrs. …","""female""",38,0,"""PC 17599""",71.2833,"""C85""","""C""",null,12:26:00,2022-08-07 01:56:33,null
3,"""Heikkinen, Mis…","""female""",26,0,"""STON/O2. 31012…",7.925,null,"""S""",1931-09-24,16:08:25,2022-08-04 20:27:37,null
4,"""Futrelle, Mrs.…","""female""",35,0,"""113803""",53.1,"""C123""","""S""",1936-11-30,null,2022-08-07 07:05:55,null
5,"""Allen, Mr. Wil…","""male""",35,0,"""373450""",8.05,null,"""S""",1918-11-07,10:59:08,2022-08-02 15:13:34,null


Now, let's check the data types of our DataFrame:

In [4]:
dict(zip(df.columns, df.dtypes))

{'PassengerId': Int64,
 'Name': Utf8,
 'Sex': Categorical,
 'Age': Int64,
 'Parch': Int64,
 'Ticket': Utf8,
 'Fare': Float64,
 'Cabin': Utf8,
 'Embarked': Categorical,
 'Birthday': Date,
 'Board time': Time,
 'Married since': Datetime(time_unit='us', time_zone=None),
 'all_NA': Utf8}

We see that most variables are now nicely specified as strings, categories, dates and ints where necessary. We can also inspect the data a bit more with `describe()`.

In [5]:
df.describe()

describe,PassengerId,Name,Sex,Age,Parch,Ticket,Fare,Cabin,Embarked,Birthday,Board time,Married since,all_NA
str,f64,str,str,f64,f64,str,f64,str,str,str,str,str,str
"""count""",891.0,"""891""","""891""",891.0,891.0,"""891""",891.0,"""891""","""891""","""891""","""891""","""891""","""891"""
"""null_count""",0.0,"""0""","""0""",177.0,0.0,"""0""",0.0,"""687""","""2""","""78""","""79""","""92""","""891"""
"""mean""",446.0,null,null,29.693277,0.381594,null,32.204208,null,null,null,null,null,null
"""std""",257.353842,null,null,14.524527,0.806057,null,49.693429,null,null,null,null,null,null
"""min""",1.0,"""Abbing, Mr. An…",null,0.0,0.0,"""110152""",0.0,"""A10""",null,"""1903-07-28""","""10:39:40""","""2022-07-15 12:…",null
"""max""",891.0,"""van Melkebeke,…",null,80.0,6.0,"""WE/P 5735""",512.3292,"""T""",null,"""1940-05-27""","""18:39:28""","""2022-08-15 10:…",null
"""median""",446.0,null,null,28.0,0.0,null,14.4542,null,null,null,null,null,null
"""25%""",223.0,null,null,20.0,0.0,null,7.8958,null,null,null,null,null,null
"""75%""",669.0,null,null,38.0,0.0,null,31.0,null,null,null,null,null,null


## Step 2: Creating a MetaFrame object from a DataFrame

Now a lot of work has already gone into creating a properly formatted DataFrame. This work pays off at this stage: let's convert the DataFrame to a MetaFrame structure with the default options. Note: this takes a little bit of time!

In [6]:
mf = MetaFrame.fit_dataframe(df)

/home/qubix/Documents/shared_work/synthetic/meta-synth/metasynth/provider.py:226: UserWarning: 
Variable PassengerId seems unique, but not set to be unique.
Set the variable to be either unique or not unique to remove this warning.

  warnings.warn(f"\nVariable {series.name} seems unique, but not set to be unique.\n"


Then, we can simply print the MetaFrame to display it in an easy-to-read format:

In [7]:
print(mf)

# Rows: 891
# Columns: 13

Column 1: "PassengerId"
- Variable Type: discrete
- Data Type: Int64
- Proportion of Missing Values: 0.0000
- Distribution:
	- Type: core.discrete_uniform
	- Provenance: builtin
	- Parameters:
		- low: 1
		- high: 892
	

Column 2: "Name"
- Variable Type: string
- Data Type: Utf8
- Proportion of Missing Values: 0.0000
- Distribution:
	- Type: core.regex
	- Provenance: builtin
	- Parameters:
		- regex: [A-Za-z]{1,12}(|[ \-'][A-Z][a-z]{4,7})[,][ ][MA-Z]((|[a][s][t][e])[r]([\.][ ][A-Z][a-z]{2,8}(|[ ][A-Z](|[a-z]{3,9}(|[ ][A-Z\(][a-z"]{1,8})))|[s][\.][ ][A-Z][a-z]{3,8}[ ]([A-Z][a-z]{4,7}[ ][\(][A-Z][a-z]{3,7}[ ][A-Z][a-z]{4,8}[\) ]|[\(][A-Z][a-z]{3,8}[ ][A-Z][a-z]{3,8}[\)]))|[i][s]{2,2}[\.][ ][A-Z][a-z]{3,8}(|[ ][A-Z][a-z]{3,8}))
	

Column 3: "Sex"
- Variable Type: categorical
- Data Type: Categorical
- Proportion of Missing Values: 0.0000
- Distribution:
	- Type: core.multinoulli
	- Provenance: builtin
	- Parameters:
		- labels: ['female' 'male']
		- probs: [0.35

Alternatively, we can preview the MetaFrame as it would be output to a file

In [8]:
json_preview = repr(mf)
print(json_preview)

{
    "n_rows": 891,
    "n_columns": 13,
    "provenance": {
        "created by": {
            "name": "MetaSynth",
            "version": "0.4.1.dev8+ga63e919.d20230816"
        },
        "creation time": "2023-09-13T14:45:04.605025"
    },
    "vars": [
        {
            "name": "PassengerId",
            "type": "discrete",
            "dtype": "Int64",
            "prop_missing": 0.0,
            "distribution": {
                "implements": "core.discrete_uniform",
                "version": "1.0",
                "provenance": "builtin",
                "class_name": "DiscreteUniformDistribution",
                "parameters": {
                    "low": 1,
                    "high": 892
                }
            }
        },
        {
            "name": "Name",
            "type": "string",
            "dtype": "Utf8",
            "prop_missing": 0.0,
            "distribution": {
                "implements": "core.regex",
                "version": "2.0",
    

## Step 3: Saving the metadata in a file

After creating the MetaFrame, we can save it to a file. The default format is `JSON`, which is both easy to read for humans and computers. This allows one to manually inspect the metadata file and verify no sensitive information would be shared. If the disclosure risk is deemed low, the JSON file can then be securely provided to others for exploratory analysis or other uses without exposing private data. 


In [9]:
# save the metadata to a file
file_path = "demonstration_metadata.json"
mf.to_json(file_path)

# you can now open and read the json file!

## Step 4: Generating synthetic data from the metadata

A previously exported MetaFrame (.json) file can be loaded into a MetaFrame object. 

In [10]:
#load previously exported MetaFrame (.json) file
mf = MetaFrame.from_json(file_path)

Once a MetaFrame is loaded, synthetic data can be generated from it. The `synthesize` method takes the number of rows to be generated as parameter and returns a DataFrame with the synthetic data.

In [11]:
# generate synthetic data
mf.synthesize(5)

PassengerId,Name,Sex,Age,Parch,Ticket,Fare,Cabin,Embarked,Birthday,Board time,Married since,all_NA
i64,str,cat,i64,i64,str,f64,str,cat,date,time,datetime[μs],f32
882,"""uhBLYhXVL, Xr.…","""male""",null,0,"""756153""",0.610886,"""K13""","""S""",1917-01-19,12:14:36,2022-08-03 09:54:15,null
205,"""jyn, Cr. Udm""","""female""",24,0,"""19449""",40.455172,null,"""S""",1908-07-29,15:48:17,2022-07-21 22:42:43,null
814,"""Jh, Er. Umvyb …","""male""",48,0,"""96094""",35.064438,null,"""C""",1935-02-26,14:14:19,2022-08-05 11:16:39,null
546,"""zOfek, Kr. Kyz…","""male""",20,0,"""7269""",50.017058,"""C71""","""S""",1922-12-02,14:14:09,null,null
11,"""pTa-Bdxwuh, Vr…","""male""",30,0,"""86867""",12.834675,null,"""S""",1905-01-26,14:35:07,2022-07-22 11:00:34,null


As you can see, the fake data looks a lot like the real data! However, it could still use some improvement. In the next sections, we will explore manual changes we can make to improve the quality of the synthetic data.

## Step 5: Improving the quality of the synthetic data

### Set unique columns

One column (PassengerId) has been detected as possibly unique by MetaSynth, as indicated by the following warning:

> "Variable PassengerId seems unique, but not set to be unique."

This column holds a variable with unique passenger identifiers, so in fact we do want synthetic data generated for this column to be unique as well. We can add this to the metadata by creating a list of options which we call a `specification`, or `spec`:

In [12]:
# First, we create a specification dictionary for the variables
var_spec = {
    "PassengerId": {"unique": True}
}

# then, we add that dictionary as the `spec` argument
mf = MetaFrame.fit_dataframe(df, spec=var_spec)

# then, let's check what the metadata about PassengerId contains!
mf["PassengerId"].to_dict()

{'name': 'PassengerId',
 'type': 'discrete',
 'dtype': 'Int64',
 'prop_missing': 0.0,
 'distribution': {'implements': 'core.unique_key',
  'version': '1.0',
  'provenance': 'builtin',
  'class_name': 'UniqueKeyDistribution',
  'parameters': {'low': 1, 'consecutive': 1}}}

So let's check what is generated from this new MetaFrame:

In [13]:
mf.synthesize(5)

PassengerId,Name,Sex,Age,Parch,Ticket,Fare,Cabin,Embarked,Birthday,Board time,Married since,all_NA
i64,str,cat,i64,i64,str,f64,f32,cat,date,time,datetime[μs],f32
1,"""awKEOdy, Piss.…","""male""",17,0,"""TJA.A. 410408""",13.313988,null,"""Q""",1911-12-26,13:09:25,null,null
2,"""bIkjxdIovjvN, …","""male""",22,0,"""9612267""",7.113703,null,"""S""",1909-11-17,16:10:29,2022-07-30 21:16:53,null
3,"""tKTVPTthhD, Nr…","""female""",47,1,"""8054""",36.150274,null,"""S""",1904-11-24,13:06:47,null,null
4,"""zR, Iiss. Lkjq…","""male""",null,0,"""9175""",5.522355,null,"""C""",1906-05-26,12:21:26,2022-08-11 19:38:25,null
5,"""VZFXHGcyDySP, …","""female""",57,0,"""24396""",14.28513,null,"""S""",1940-05-21,17:00:47,2022-07-30 20:57:49,null


Now we that the `PassengerId` column is correctly represented with increasing id numbers.

### Fake names (and others)

As one can see, the `Name` of the passengers is not quite so well synthesized. The reason is that the string type interpreter in MetaSynth is designed for `structured` strings (like room numbers such as `B1.09`, `B1.01` or `A1.08`) and not unstructured strings. However, MetaSynth supports the [faker](https://faker.readthedocs.io/en/master/index.html) package, which includes a lot of data types that it can fake. The columns using faker are not based on the real data at all so they do not disclose any info about the real data.

We fake names as follows:

In [14]:
# First, we create a specification dictionary for the variables
from metasyn.distribution import FakerDistribution

var_spec = {
    "PassengerId": {"unique": True}, 
    "Name": {"distribution": FakerDistribution("name")}
}

mf = MetaFrame.fit_dataframe(df, spec=var_spec)
mf.synthesize(5)

PassengerId,Name,Sex,Age,Parch,Ticket,Fare,Cabin,Embarked,Birthday,Board time,Married since,all_NA
i64,str,cat,i64,i64,str,f64,f32,cat,date,time,datetime[μs],f32
1,"""Michael Morale…","""male""",27,0,"""9256""",52.678668,null,"""S""",1925-10-10,10:40:23,2022-08-14 08:16:50,null
2,"""Barbara Curtis…","""female""",22,0,"""399720""",185.674337,null,"""S""",1910-11-22,16:33:16,2022-07-28 04:42:43,null
3,"""Kelsey Sanchez…","""male""",24,0,"""ZR.A. 8795835""",20.583335,null,"""S""",1933-01-26,null,2022-08-11 17:15:46,null
4,"""Dr. Glen Woods…","""male""",1,2,"""3967698""",99.910127,null,"""S""",1914-11-23,10:51:31,2022-07-28 02:39:19,null
5,"""Monica Kirby""","""female""",2,0,"""573728""",18.63288,null,"""S""",1935-02-02,11:08:08,2022-07-17 01:43:02,null


That already looks a lot better for the `Name` column!

### Set distributions manually

Without user input, the distribution chosen for each variable is inferred by choosing the best fitting from available distributions for the variable type. However, we can also manually specify which distribution to fit, or we can even just fully specify how the variable should be generated.

In [15]:
from metasyn.distribution import DiscreteUniformDistribution

var_spec = {
    "PassengerId": {"unique": True}, 
    "Name": {"distribution": FakerDistribution("name")},
    "Fare": {"distribution": "LogNormalDistribution"}, # estimate / fit an exponential distribution based on the data
    "Age": {"distribution": DiscreteUniformDistribution(20, 40)} # fully specify a distribution for age (uniform between 20 and 40)
}

mf = MetaFrame.fit_dataframe(df, spec=var_spec)
mf.synthesize(5)

PassengerId,Name,Sex,Age,Parch,Ticket,Fare,Cabin,Embarked,Birthday,Board time,Married since,all_NA
i64,str,cat,i64,i64,str,f64,str,cat,date,time,datetime[μs],f32
1,"""Michael Morale…","""male""",21,0,"""0830164""",4.594758,"""N6""","""S""",null,10:42:12,2022-08-11 05:48:20,null
2,"""Barbara Curtis…","""female""",null,0,"""08899""",0.65618,"""G49""","""S""",1905-01-25,14:27:45,2022-07-30 10:46:53,null
3,"""Kelsey Sanchez…","""female""",null,0,"""146849""",0.977851,null,"""S""",1908-02-06,17:30:38,2022-08-07 20:54:02,null
4,"""Dr. Glen Woods…","""male""",26,0,"""70979""",0.166117,null,"""S""",1935-01-21,12:24:53,2022-07-30 12:50:23,null
5,"""Monica Kirby""","""male""",null,1,"""57167""",0.215189,null,"""C""",1922-06-22,14:59:04,2022-07-23 14:28:38,null


### Specifying the distribution of structured strings

For more or less structured strings, we can manually set the structure of the strings based on regular expressions. For example, we see that most Cabins are structured like [A-F] and then 2 or 3 digit numbers. We can include this as follows:

In [16]:
from metasyn.distribution import RegexDistribution

# To create a regex distribution, you need a list of tuples, where each tuple is an element.
# The first part of the tuple is a string representation of the regex, while the second is the proportion of the
# time the regex element is used.
cabin_distribution = RegexDistribution(r"[ABCDEF][0-9]{2,3}")  # Add the r so that it becomes a literal string.
# just for completeness: data generated from this distribution will always match the regex [ABCDEF]?(\d{2,3})?

var_spec = {
    "PassengerId": {"unique": True}, 
    "Name": {"distribution": FakerDistribution("name")},
    "Fare": {"distribution": "ExponentialDistribution"}, # estimate / fit an exponential distribution based on the data
    "Age": {"distribution": DiscreteUniformDistribution(20, 40)}, # fully specify a distribution for age (uniform between 20 and 40)
    "Cabin": {"distribution": cabin_distribution}
}

mf = MetaFrame.fit_dataframe(df, spec=var_spec)
mf.synthesize(10)

PassengerId,Name,Sex,Age,Parch,Ticket,Fare,Cabin,Embarked,Birthday,Board time,Married since,all_NA
i64,str,cat,i64,i64,str,f64,str,cat,date,time,datetime[μs],f32
1,"""Michael Morale…","""male""",null,0,"""3214327""",26.522736,null,"""C""",1920-05-30,12:46:31,2022-08-08 09:00:13,null
2,"""Barbara Curtis…","""male""",32,0,"""8947447""",8.91739,"""B608""","""S""",1928-11-19,12:28:05,2022-08-10 14:46:09,null
3,"""Kelsey Sanchez…","""male""",37,0,"""001637""",17.523508,"""C712""","""S""",1931-04-04,17:31:09,2022-08-01 09:46:11,null
4,"""Dr. Glen Woods…","""female""",39,0,"""381209""",45.129448,null,"""C""",1937-04-09,14:48:35,2022-07-30 20:46:26,null
5,"""Monica Kirby""","""male""",null,0,"""1789""",9.833569,"""E972""","""S""",1912-01-14,16:58:44,2022-07-27 22:27:21,null
6,"""Tara Riley""","""male""",27,0,"""91105""",41.753713,null,"""S""",1928-12-28,18:01:01,2022-08-01 00:50:18,null
7,"""Joseph Bruce""","""male""",32,0,"""652107""",9.175778,null,"""Q""",1908-12-23,14:59:54,2022-07-16 08:22:50,null
8,"""Connie Alvarez…","""male""",null,0,"""49345""",57.769254,null,"""S""",null,null,2022-07-19 23:32:34,null
9,"""Jacob Graves""","""male""",null,1,"""PC 19329""",38.391015,null,"""S""",1922-09-18,18:28:51,2022-08-14 22:41:01,null


## Comparing the final synthetic dataset to the original

Let's first compare the averages of the numerical columns:

In [17]:
df.mean()

PassengerId,Name,Sex,Age,Parch,Ticket,Fare,Cabin,Embarked,Birthday,Board time,Married since,all_NA
f64,str,cat,f64,f64,str,f64,str,cat,date,time,datetime[μs],str
446.0,null,null,29.693277,0.381594,null,32.204208,null,null,null,null,null,null


In [18]:
mf.synthesize(len(df)).mean()

PassengerId,Name,Sex,Age,Parch,Ticket,Fare,Cabin,Embarked,Birthday,Board time,Married since,all_NA
f64,str,cat,f64,f64,str,f64,str,cat,date,time,datetime[μs],f32
446.0,null,null,29.23736,0.329966,null,33.536027,null,null,null,null,null,null


Then, we can also see how many missing values are in each column

In [19]:
df.null_count()

PassengerId,Name,Sex,Age,Parch,Ticket,Fare,Cabin,Embarked,Birthday,Board time,Married since,all_NA
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
0,0,0,177,0,0,0,687,2,78,79,92,891


In [20]:
mf.synthesize(len(df)).null_count()

PassengerId,Name,Sex,Age,Parch,Ticket,Fare,Cabin,Embarked,Birthday,Board time,Married since,all_NA
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
0,0,0,182,0,0,0,705,0,87,83,93,891
